In [1]:
from pyspark.sql import SparkSession

In [2]:
# 1. Создаем сессию Spark
spark = SparkSession.builder \
    .appName("SensorLogs") \
    .getOrCreate()

In [3]:
# 2. Создаем RDD из sensor_logs.txt
rdd = spark.sparkContext.textFile("sensor_logs.txt")

In [4]:
# 3. Разделим текст на строки, используя разделитель '\n' 
lines_rdd = rdd.flatMap(lambda line: line.split('\n'))

# 4. Разделим каждую строку на отдельные поля, используя символ | как разделитель
fields_rdd = lines_rdd.map(lambda line: line.split('|'))

In [5]:
# Выведем первые 5 записей для проверки
print("Первые 5 записей:")
for record in fields_rdd.take(5):
    print(record)

Первые 5 записей:
['2025-01-10 08:00:05', 'S101', '25.3', '10.1', 'OK', 'NULL']
['2025-01-10 08:05:10', 'S102', '26.1', 'NULL', 'WARNING', 'E01']
['2025-01-10 08:10:00', 'S101', '25.5', '10.2', 'OK', 'NULL']
['2025-01-10 08:15:30', 'S103', 'NULL', '9.9', 'ERROR', 'E05']
['2025-01-10 08:20:15', 'S102', '25.8', '10.0', 'OK', 'NULL']


In [6]:
# 5. Преобразуем поля (temperature, pressure) в соответствующий числовой тип (float). 
# Преобразуем NULL значения в None
def parse_fields(fields):
    try:
        temperature = float(fields[2]) if fields[2] != 'NULL' else None
        pressure = float(fields[3]) if fields[3] != 'NULL' else None
        status = fields[4]
        sensor_id = fields[1]
        error_code = fields[5] if fields[5] != 'NULL' else None
        timestamp = fields[0]
        return (status, sensor_id, temperature, pressure, error_code, timestamp)
    except ValueError:
        return None
    
parsed_rdd = fields_rdd.map(parse_fields)

In [7]:
# 6. Посчитаем, сколько всего записей (активностей) было получено для каждого status (например, OK: X, WARNING: Y, ERROR: Z)
status_count_rdd = parsed_rdd \
    .filter(lambda x: x is not None) \
    .map(lambda x: (x[0], 1)) \
    .reduceByKey(lambda a, b: a + b)

In [8]:
# 7. Определим sensor_id, которые сообщают о статусе ERROR и количество полученных ошибок каждым из них.
error_count_rdd = parsed_rdd \
    .filter(lambda x: x is not None and x[0] == 'ERROR') \
    .map(lambda x: (x[1], 1)) \
    .reduceByKey(lambda a, b: a + b)

In [9]:
# 8. Рассчитаем среднюю temperature. 
# Исключим записи без температуры из расчета среднего. 
# Округлим до двух знаков после запятой.
average_temp_rdd = parsed_rdd \
    .filter(lambda x: x is not None and x[2] is not None) \
    .map(lambda x: (1, (x[2], 1))) \
    .reduceByKey(lambda a, b: (a[0] + b[0], a[1] + b[1])) \
    .mapValues(lambda x: round(x[0] / x[1], 2))

In [10]:
# 9. Для каждого error_code (если он не NULL), посчитаем общее количество его появлений. 
# Отсортируем результат по номеру кода.
error_code_count_rdd = parsed_rdd \
    .filter(lambda x: x is not None and x[4] is not None) \
    .map(lambda x: (x[4], 1)) \
    .reduceByKey(lambda a, b: a + b) \
    .sortByKey()

In [11]:
# 10. Отфильтруем RDD, чтобы получить только те записи, где pressure выше 10.0 и temperature выше 26.0.  
# Для каждой отфильтрованной записи выведем кортеж (timestamp, sensor_id, temperature, pressure).
filtered_rdd = parsed_rdd \
    .filter(lambda x: x is not None and x[2] is not None and x[3] is not None and x[3] > 10.0 and x[2] > 26.0) \
    .map(lambda x: (x[5], x[1], x[2], x[3]))

In [12]:
# 11. Вывод всех результатов из шагов 6-10
def print_rdd(rdd, title):
    print(f"{title}:")
    for item in rdd.collect():
        print(item)
    print()

In [13]:
print_rdd(status_count_rdd, "Status Count")

Status Count:
('WARNING', 6)
('OK', 28)
('ERROR', 6)



In [14]:
print_rdd(error_count_rdd, "Error Count")

Error Count:
('S105', 1)
('S103', 2)
('S104', 3)



In [15]:
print(f"Average Temperature: {average_temp_rdd.collect()[0][1]}")

Average Temperature: 25.84


In [16]:
print_rdd(error_code_count_rdd, "Error Code Count")

Error Code Count:
('E01', 2)
('E02', 2)
('E03', 2)
('E04', 2)
('E05', 2)



In [17]:
print_rdd(filtered_rdd, "Filtered Records (Pressure > 10.0 and Temperature > 26.0)")

Filtered Records (Pressure > 10.0 and Temperature > 26.0):
('2025-01-10 08:30:20', 'S104', 27.0, 10.3)
('2025-01-11 09:00:00', 'S104', 27.2, 10.4)
('2025-01-12 10:15:00', 'S102', 26.3, 10.2)
('2025-01-13 11:00:00', 'S104', 27.3, 10.6)
('2025-01-14 08:40:15', 'S101', 26.1, 10.6)
('2025-01-14 08:45:00', 'S102', 26.4, 10.4)
('2025-01-15 09:00:00', 'S104', 27.5, 10.7)
('2025-01-15 09:10:00', 'S101', 26.2, 10.7)
('2025-01-16 10:00:10', 'S104', 27.6, 10.8)
('2025-01-16 10:10:15', 'S101', 26.3, 10.8)
('2025-01-16 10:15:00', 'S102', 26.6, 10.5)

